In [74]:
train_test_split=0.7
train_valid_split=0.7 #after augmentation

In [48]:
import os, os.path
import re
import pandas as pd
import numpy as np
import cv2
import requests
from PIL import Image
from io import BytesIO
from imageio import imread
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
import tensorflow as tf
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import backend as keras
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import shutil
import time

data=[([],[]),([],[]),([],[])]

count_temp=0

directory="D:/tumor/final_dataset/"
subdir=["benign_image/","benign_mask/","malignant_image/","malignant_mask/","normal_image/","normal_mask/"]

In [55]:
dataset=['dataset/','train/','test/','validation/']

In [72]:
def LoadData(directory,subdir):
    for data in dataset:
        if data=='dataset/':
            if os.path.exists(data):
                shutil.rmtree(data)
            os.makedirs(data)
        else:
            if os.path.exists('dataset/'+data):
                shutil.rmtree('dataset/'+data)
            os.makedirs('dataset/'+data)
            for sub in subdir:
                if os.path.exists('dataset/'+data+sub):
                    shutil.rmtree('dataset/'+data+sub)
                os.makedirs('dataset/'+data+sub)
    
    i=0
    for sub in subdir:
        print(f"{sub[:-1]} : {len(os.listdir(directory+subdir[i]))}")
        print()
        i+=1
        
        

In [73]:
LoadData(directory,subdir)

benign_image : 437

benign_mask : 437

malignant_image : 210

malignant_mask : 210

normal_image : 133

normal_mask : 133



In [2]:
def LoadData(imgPath,subd,ix,images,masked,images_url,masked_url,check):
    global count_temp
    count_temp+=1
    
    if(count_temp>3):
        for i in range(3):
            data[i][0].clear()
            data[i][1].clear()
        count_temp=1
        
    imgPath=imgPath+subd
    imgNames = os.listdir(imgPath)
    
    for item in imgNames:
        img = plt.imread(imgPath + item)
        if item.endswith(')_mask resized_greyscale.jpg'):
            masked.append(img)
        elif item.endswith(') resized.jpg'):
            images.append(img)
        elif item.endswith(' resized_greyscale.jpg'):
            masked[-1]=np.add(masked[-1], img)

def LoadData_Info_with_print():
    
    print("Information about the data : \n")
    
    for i in range(3):
        LoadData(directory,subdir[i],i,data[i][0],data[i][1],data_url[i][0],data_url[i][1],1)

    print(f"\nTotal images : {len(data[0][0])+len(data[1][0])+len(data[2][0])}")
    for i in range(3):
        print(len(data[i][0]))
    print()
    
    print(f"Total masked : {len(data[0][1])+len(data[1][1])+len(data[2][1])}")
    for i in range(3):
        print(len(data[i][1]))
    print()
    
def LoadData_Info():
    for i in range(3):
        LoadData(directory,subdir[i],i,data[i][0],data[i][1],data_url[i][0],data_url[i][1],0)

In [3]:
def UNet_data_with_valid(train_ratio,test_ratio,valid_ratio): # 0.7,0.15,0.15
    LoadData_Info()

    l=[]

    test_ratio  = 1 - train_ratio
    valid_ratio = valid_ratio/test_ratio # 0.15/0.3=0.5
    test_ratio  = 1 - valid_ratio
    
    for i in range(3):
        X_train, X_rem , y_train, y_rem =train_test_split(data[i][0], data[i][1], train_size=train_ratio, random_state=42)
        X_valid, X_test, y_valid, y_test=train_test_split(X_rem,y_rem, test_size=test_ratio,random_state=42)
        l.append([X_train, y_train, X_valid, y_valid, X_test, y_test])

    X_train  = [] ; y_train  = [] ; X_valid  = [] ; y_valid  = [] ; X_test   = [] ; y_test   = []
    
    for i in range(3):
        X_train.append(l[i][0]) ; y_train.append(l[i][1])
        X_valid.append(l[i][2]) ; y_valid.append(l[i][3])
        X_test.append(l[i][4])  ; y_test.append(l[i][5])

    X_train=np.concatenate(X_train, axis=0) ; X_test=np.concatenate(X_test, axis=0) ; X_valid=np.concatenate(X_valid, axis=0)
    y_train=np.concatenate(y_train, axis=0) ; y_test=np.concatenate(y_test, axis=0) ; y_valid=np.concatenate(y_valid, axis=0)
        
    X_train,y_train=shuffle(X_train, y_train, random_state=0)
    X_test,y_test=shuffle(X_test, y_test, random_state=0)
    X_valid,y_valid=shuffle(X_valid, y_valid, random_state=0)   

    return [X_train, X_valid, X_test, y_train, y_valid, y_test]

In [5]:
LoadData_Info_with_print()

X_train, X_valid, X_test, y_train, y_valid, y_test = UNet_data_with_valid(0.7,0.1,0.2)
X_train=(X_train/255).astype(np.float32)
X_valid=(X_valid/255).astype(np.float32)
X_test=(X_test/255).astype(np.float32)
y_train=(y_train/255).astype(np.float32)
y_valid=(y_valid/255).astype(np.float32)
y_test=(y_test/255).astype(np.float32)

print(len(X_train),len(X_valid),len(X_test),len(y_train),len(y_valid),len(y_test))

Information about the data : 


Total images : 780
437
210
133

Total masked : 780
437
210
133

545 154 81 545 154 81


In [12]:
##UNET MODEL :

def conv_block(input_, num_filters):
    
#     conv2D_1 = Conv2D(filters = num_filters,kernel_size =  3, padding="same")(input_)
    conv2D_1 = Conv2D(filters = num_filters,kernel_size =  3, kernel_initializer = 'he_normal', padding="same")(input_)
    batch1 = BatchNormalization()(conv2D_1)
    act1 = Activation("relu")(batch1)

#     conv2D_2 = Conv2D(filters = num_filters,kernel_size =  3, padding="same")(act1)
    conv2D_2 = Conv2D(filters = num_filters,kernel_size =  3, kernel_initializer = 'he_normal', padding="same")(act1)
    batch2 = BatchNormalization()(conv2D_2)
    act2 = Activation("relu")(batch2)

    return act2

# count=0

def encoder_block(input_, num_filters):
#     global count
#     count+=1
#     print(count)
    conv = conv_block(input_, num_filters)
#     if count==4:
#         drop = Dropout(0.075)(conv)
#         pool = MaxPool2D((2, 2))(drop)
#         count=0
#         return conv, pool
    pool = MaxPool2D((2, 2))(conv)
    drop = Dropout(0.075)(pool)
    return conv, drop

def decoder_block(input_, skip_features, num_filters):
    x = Conv2DTranspose(filters = num_filters,kernel_size = (2, 2), strides=2, padding="same")(input_)
    x = Concatenate()([x, skip_features])
    x = Dropout(0.075)(x)
    x = conv_block(x, num_filters)
    return x

def unet_build(input_shape):
    
    inputs = Input(input_shape)
    
    conv1, pool1 = encoder_block(inputs, 16)
    conv2, pool2 = encoder_block(pool1, 32)
    conv3, pool3 = encoder_block(pool2, 64) 
    conv4, pool4 = encoder_block(pool3, 128) 

    bridge = conv_block(pool4, 256)

    decoder_1 = decoder_block(bridge, conv4, 128)
    decoder_2 = decoder_block(decoder_1, conv3, 64)
    decoder_3 = decoder_block(decoder_2, conv2, 32)
    decoder_4 = decoder_block(decoder_3, conv1, 16)

    outputs = Conv2D(1, (1,1), padding="same", activation="sigmoid") (decoder_4)
#     outputs = Conv2D(1, 1, padding="same") (decoder_4)

    model = Model(inputs, outputs, name="U-Net")
    return model

def improvised_unet_build(input_shape):
    
    inputs = Input(input_shape)
    
    conv1, pool1 = encoder_block(inputs, 16)
    conv2, pool2 = encoder_block(pool1, 32)
    conv3, pool3 = encoder_block(pool2, 64) 
    conv4, pool4 = encoder_block(pool3, 128) 

    bridge = conv_block(pool4, 256)

    decoder_1 = decoder_block(bridge, conv4, 128)
    decoder_2 = decoder_block(decoder_1, conv3, 64)
    decoder_3 = decoder_block(decoder_2, conv2, 32)
    
#     decoder_4 = decoder_block(decoder_3, conv1, 16)

    x = Conv2DTranspose(filters = 16,kernel_size = (2, 2), strides=2, padding="same")(decoder_3)
    x = Concatenate()([x, conv1])
    x = Dropout(0.075)(x)
    
    x1 = conv_block(x, 16)
    y1_output = Conv2D(1, 1, padding="same", activation="sigmoid",name='grayscale') (x1)

    x2 = conv_block(x, 16)
    y2_output = Conv2D(3, 1, padding="same", activation="sigmoid",name='colour') (x2)
    
    model = Model(inputs=inputs, outputs=[y1_output, y2_output], name="U-Net")
    return model
#     outputs = Conv2D(1, 1, padding="same") (decoder_4)

def DiceAccuracy(targets, inputs, smooth=1e-6):
    
    #flatten label and prediction tensors
    inputs = keras.flatten(inputs)
    targets = keras.flatten(targets)
    
    intersection = keras.sum(targets*inputs,keepdims=True)
    dice = (2*intersection + smooth) / (keras.sum(targets) + keras.sum(inputs) + smooth)
    return dice

In [13]:
input_shape = (256, 256, 3)
unet_model = unet_build(input_shape)
unet_model.summary()

input_shape = (256, 256, 3)
improvised_unet_model = improvised_unet_build(input_shape)
# improvised_unet_model.summary()

Model: "U-Net"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 256, 256, 16) 448         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 256, 256, 16) 64          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 256, 256, 16) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [14]:
unet_model.compile(optimizer='adam', loss='binary_crossentropy',metrics=[DiceAccuracy])

# improvised_unet_model.compile(optimizer='adam',
#                               loss={'grayscale': 'binary_crossentropy', 'colour': 'mse'},
#                               metrics={'grayscale': DiceAccuracy, 'colour': tf.keras.metrics.Accuracy()},
#                               )

In [29]:
filepath='_BCE-1e-3-x0_new'
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_DiceAccuracy',mode='max',verbose=1,save_best_only=True)
callbacks = [checkpoint]

In [30]:
history = unet_model.fit(X_train,y_train,batch_size=15,epochs=100,validation_data=(X_valid,y_valid),callbacks=callbacks)
# history = improvised_unet_model.fit(X_train,(y_train,X_train) ,batch_size=25,epochs=300,validation_data=(X_valid,(y_valid,X_valid)),callbacks=callbacks)

Epoch 1/100
37/37 [==============================] - 6s 165ms/step - loss: 0.1112 - DiceAccuracy: 0.6038 - val_loss: 0.1308 - val_DiceAccuracy: 0.5290

Epoch 00001: val_DiceAccuracy improved from -inf to 0.52903, saving model to _BCE-1e-3-x0_new
INFO:tensorflow:Assets written to: _BCE-1e-3-x0_new\assets
Epoch 2/100
37/37 [==============================] - 7s 169ms/step - loss: 0.1060 - DiceAccuracy: 0.6165 - val_loss: 0.2615 - val_DiceAccuracy: 0.4307

Epoch 00002: val_DiceAccuracy did not improve from 0.52903
Epoch 3/100
37/37 [==============================] - 6s 168ms/step - loss: 0.0969 - DiceAccuracy: 0.6483 - val_loss: 0.1390 - val_DiceAccuracy: 0.4751

Epoch 00003: val_DiceAccuracy did not improve from 0.52903
Epoch 4/100
37/37 [==============================] - 6s 168ms/step - loss: 0.0959 - DiceAccuracy: 0.6512 - val_loss: 0.2855 - val_DiceAccuracy: 0.4379

Epoch 00004: val_DiceAccuracy did not improve from 0.52903
Epoch 5/100
37/37 [==============================] - 6s 168ms/

37/37 [==============================] - 6s 167ms/step - loss: 0.0326 - DiceAccuracy: 0.8693 - val_loss: 0.1735 - val_DiceAccuracy: 0.6187

Epoch 00035: val_DiceAccuracy did not improve from 0.65287
Epoch 36/100
37/37 [==============================] - 6s 167ms/step - loss: 0.0334 - DiceAccuracy: 0.8736 - val_loss: 0.1641 - val_DiceAccuracy: 0.6475

Epoch 00036: val_DiceAccuracy did not improve from 0.65287
Epoch 37/100
37/37 [==============================] - 6s 167ms/step - loss: 0.0379 - DiceAccuracy: 0.8600 - val_loss: 0.1829 - val_DiceAccuracy: 0.6042

Epoch 00037: val_DiceAccuracy did not improve from 0.65287
Epoch 38/100
37/37 [==============================] - 6s 167ms/step - loss: 0.0319 - DiceAccuracy: 0.8732 - val_loss: 0.1631 - val_DiceAccuracy: 0.6352

Epoch 00038: val_DiceAccuracy did not improve from 0.65287
Epoch 39/100
37/37 [==============================] - 6s 167ms/step - loss: 0.0317 - DiceAccuracy: 0.8759 - val_loss: 0.1700 - val_DiceAccuracy: 0.6158

Epoch 00039:

Epoch 71/100
37/37 [==============================] - 6s 167ms/step - loss: 0.0163 - DiceAccuracy: 0.9349 - val_loss: 0.2021 - val_DiceAccuracy: 0.6473

Epoch 00071: val_DiceAccuracy did not improve from 0.67087
Epoch 72/100
37/37 [==============================] - 6s 167ms/step - loss: 0.0154 - DiceAccuracy: 0.9410 - val_loss: 0.1968 - val_DiceAccuracy: 0.6665

Epoch 00072: val_DiceAccuracy did not improve from 0.67087
Epoch 73/100
37/37 [==============================] - 6s 167ms/step - loss: 0.0162 - DiceAccuracy: 0.9380 - val_loss: 0.1940 - val_DiceAccuracy: 0.6768

Epoch 00073: val_DiceAccuracy improved from 0.67087 to 0.67676, saving model to _BCE-1e-3-x0_new
INFO:tensorflow:Assets written to: _BCE-1e-3-x0_new\assets
Epoch 74/100
37/37 [==============================] - 6s 167ms/step - loss: 0.0159 - DiceAccuracy: 0.9385 - val_loss: 0.2018 - val_DiceAccuracy: 0.6672

Epoch 00074: val_DiceAccuracy did not improve from 0.67676
Epoch 75/100
37/37 [==============================] - 6

In [31]:
model_best = load_model(filepath,custom_objects={'DiceAccuracy':DiceAccuracy,})

In [32]:
data_saves=[(X_test,y_test),(X_train,y_train),(X_valid,y_valid)]
dirs=[('/test/test_tumor/','/test/test_true/','/test/test_predicted/'),
      ('/train/train_tumor/','/train/train_true/','/train/train_predicted/'),
      ('/valid/valid_tumor/','/valid/valid_true/','/valid/valid_predicted/')]
parent=['results','train','test','valid']

In [33]:
for x in range(len(parent)):
    if(x==0):
        if os.path.exists(parent[x]):
            shutil.rmtree(parent[x])
        os.makedirs(parent[x])
    else:
        if os.path.exists(parent[0]+"/"+parent[x]):
            shutil.rmtree(parent[0]+"/"+parent[x])
        os.makedirs(parent[0]+"/"+parent[x])

In [34]:
for i in range(3):
    for j in range(3):
        if os.path.exists("results"+dirs[i][j]):
            shutil.rmtree("results"+dirs[i][j])
        os.makedirs("results"+dirs[i][j])
        
        if(j==0):
            for k in range(len(data_saves[i][0])):
                plt.imsave("results"+dirs[i][j]+str(k+1)+".jpeg", data_saves[i][0][k])
        if(j==1):
            for k in range(len(data_saves[i][1])):
                plt.imsave("results"+dirs[i][j]+str(k+1)+".jpeg", (data_saves[i][1][k] * 255).astype(np.uint8).reshape(256, 256))
        if(j==2):
            pred=model_best.predict(data_saves[i][0])
            for k in range(len(pred)):
                plt.imsave("results"+dirs[i][j]+str(k+1)+".jpeg", (pred[k] * 255).astype(np.uint8).reshape(256, 256))

In [42]:
DiceAccuracy(model_best.predict(X_test), y_test, smooth=1e-6).numpy()[0]

0.7115872